In [1]:
import numpy as np
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
train_data = TabularDataset(data=r"resources\bike-sharing-demand\train.csv")
test_data = TabularDataset(data=r"resources\bike-sharing-demand\test.csv")
submission = pd.read_csv(r"resources\bike-sharing-demand\sampleSubmission.csv")

In [3]:
# basic feature engineering
train_data['datetime'] = pd.to_datetime(train_data['datetime'])
test_data['datetime'] = pd.to_datetime(test_data['datetime'])

train_data['hour'] = train_data['datetime'].dt.hour
train_data['dayofweek'] = train_data['datetime'].dt.dayofweek
train_data['month'] = train_data['datetime'].dt.month
train_data['is_rush_hour'] = train_data['hour'].isin([7, 8, 9, 16, 17, 18, 19])
train_data['is_rush_hour'] = (train_data['is_rush_hour'] & (train_data['workingday'] == 1)).astype(int)

test_data['hour'] = test_data['datetime'].dt.hour
test_data['dayofweek'] = test_data['datetime'].dt.dayofweek
test_data['month'] = test_data['datetime'].dt.month
test_data['is_rush_hour'] = test_data['hour'].isin([7, 8, 9, 16, 17, 18, 19])
test_data['is_rush_hour'] = (test_data['is_rush_hour'] & (test_data['workingday'] == 1)).astype(int)


train_data['casual_log'] = np.log1p(train_data['casual'])
train_data['registered_log'] = np.log1p(train_data['registered'])

In [4]:
# lets build 2 models, one for casual and one for registered, then we can sum the predictions to get the count
# we will first drop some columns

casual_train_data = train_data.copy()
registered_train_data = train_data.copy()

casual_train_data.drop(columns=['count', 'registered', 'casual', 'datetime', 'atemp', 'registered_log'], inplace=True)
registered_train_data.drop(columns=['count', 'registered', 'casual', 'datetime', 'atemp', 'casual_log'], inplace=True)

In [5]:
casual_label = "casual_log"
registered_label = "registered_log"

In [6]:
# build casual model
casual_predictor = TabularPredictor(label=casual_label, problem_type='regression', path="casual_1").fit(casual_train_data, presets="best", time_limit=60*10)

Preset alias specified: 'best' maps to 'best_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.1
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26200
CPU Count:          16
Pytorch Version:    2.9.1+cpu
CUDA Version:       CUDA is not available
Memory Avail:       1.00 GB / 7.36 GB (13.6%)
Disk Space Avail:   27.03 GB / 475.83 GB (5.7%)
Presets specified: ['best']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `

(_ray_fit pid=27916) [1000]	valid_set's rmse: 0.51143


(_dystack pid=18904) 	-0.4835	 = Validation score   (-root_mean_squared_error)
(_dystack pid=18904) 	8.61s	 = Training   runtime
(_dystack pid=18904) 	1.25s	 = Validation runtime
(_dystack pid=18904) Fitting model: LightGBM_BAG_L1 ... Training model for up to 69.07s of the 114.84s of remaining time.
(_dystack pid=18904) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=2, gpus=0, memory=1.65%)


(_ray_fit pid=20568) [1000]	valid_set's rmse: 0.49494 [repeated 17x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


(_dystack pid=18904) 	-0.4784	 = Validation score   (-root_mean_squared_error)
(_dystack pid=18904) 	3.5s	 = Training   runtime
(_dystack pid=18904) 	0.33s	 = Validation runtime
(_dystack pid=18904) Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 56.79s of the 102.55s of remaining time.
(_dystack pid=18904) 	Fitting 1 model on all data (use_child_oof=True) | Fitting with cpus=16, gpus=0, mem=0.0/0.9 GB
(_dystack pid=18904) 	Warning: Reducing model 'n_estimators' from 300 -> 184 due to low memory. Expected memory usage reduced from 24.36% -> 15.0% of available memory...
(_dystack pid=18904) 	-0.4988	 = Validation score   (-root_mean_squared_error)
(_dystack pid=18904) 	1.8s	 = Training   runtime
(_dystack pid=18904) 	0.46s	 = Validation runtime
(_dystack pid=18904) Fitting model: CatBoost_BAG_L1 ... Training model for up to 54.21s of the 99.98s of remaining time.
(_dystack pid=18904) 	Memory not enough to fit 8 folds in parallel. Will train 4 folds in parallel instead

In [7]:
# build registered model
registered_predictor = TabularPredictor(label=registered_label, problem_type='regression', path="registered_1").fit(registered_train_data, presets="best", time_limit=60*10)

Preset alias specified: 'best' maps to 'best_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.1
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26200
CPU Count:          16
Pytorch Version:    2.9.1+cpu
CUDA Version:       CUDA is not available
Memory Avail:       1.77 GB / 7.36 GB (24.0%)
Disk Space Avail:   19.22 GB / 475.83 GB (4.0%)
Presets specified: ['best']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `

KeyboardInterrupt: 

In [33]:
registered_predictor.feature_importance(registered_train_data)

Computing feature importance via permutation shuffling for 10 features using 5000 rows with 5 shuffle sets...
	1259.93s	= Expected runtime (251.99s per shuffle set)
	677.3s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
hour,1.634822,0.014819,8.101169e-10,5,1.665334,1.604309
datetime,0.454606,0.006384,4.666929e-09,5,0.467751,0.441460
workingday,0.305905,0.008310,6.528947e-08,5,0.323016,0.288794
humidity,0.090055,0.002341,5.473967e-08,5,0.094875,0.085235
weather,0.071035,0.000619,6.899918e-10,5,0.072309,0.069762
temp,0.070144,0.001751,4.657564e-08,5,0.073750,0.066538
atemp,0.044243,0.001089,4.393451e-08,5,0.046484,0.042001
windspeed,0.026911,0.001340,7.358690e-07,5,0.029671,0.024151
season,0.019866,0.000384,1.675779e-08,5,0.020656,0.019075
holiday,0.009761,0.000821,5.942105e-06,5,0.011450,0.008071


In [ ]:
casual_predictor.feature_importance(casual_train_data)

Computing feature importance via permutation shuffling for 10 features using 5000 rows with 5 shuffle sets...


	808.11s	= Expected runtime (161.62s per shuffle set)
	285.09s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
hour,1.244051,0.012228,1.119967e-09,5,1.269229,1.218873
datetime,0.325759,0.003622,1.834428e-09,5,0.333218,0.318301
temp,0.320523,0.004040,3.027268e-09,5,0.328841,0.312206
workingday,0.206686,0.007071,1.641311e-07,5,0.221246,0.192127
humidity,0.133197,0.003875,8.587677e-08,5,0.141176,0.125218
atemp,0.101050,0.003099,1.059533e-07,5,0.107430,0.094670
weather,0.060829,0.003298,1.032534e-06,5,0.067619,0.054039
season,0.040902,0.000964,3.693006e-08,5,0.042886,0.038918
windspeed,0.035092,0.001088,1.106089e-07,5,0.037331,0.032852
holiday,0.004248,0.000340,4.896779e-06,5,0.004949,0.003548


In [ ]:
print("Predicting Casual...")
# Remember: The model outputs log1p(casual)
pred_casual_log = casual_predictor.predict(test_data)

print("Predicting Registered...")
pred_registered_log = registered_predictor.predict(test_data)

# 3. Invert the Log Transform (expm1 = exp(x) - 1)
pred_casual = np.expm1(pred_casual_log)
pred_registered = np.expm1(pred_registered_log)

# 4. Safety Clip (Models can theoretically predict negative numbers)
pred_casual = np.maximum(pred_casual, 0)
pred_registered = np.maximum(pred_registered, 0)

Predicting Casual...
Predicting Registered...


In [ ]:
final_count = pred_casual + pred_registered

# 6. Save Submission
submission = pd.read_csv(r"resources\bike-sharing-demand\sampleSubmission.csv")
submission['count'] = final_count
submission.to_csv("submission_autogluon_best_quality.csv", index=False)

print("Saved submission_autogluon_best_quality.csv")

Saved submission_autogluon_best_quality.csv


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_log_error

def evaluate_on_train(casual_predictor, registered_predictor, train_data):
    """
    Calculates the combined RMSLE score on the training data.
    """
    print("Generating predictions on Training Set...")
    
    # 1. Predict (Output is in Log Space)
    # AutoGluon automatically ignores the target column if present
    pred_casual_log = casual_predictor.predict(train_data)
    pred_registered_log = registered_predictor.predict(train_data)
    
    # 2. Inverse Transform (Log -> Count)
    pred_casual = np.expm1(pred_casual_log)
    pred_registered = np.expm1(pred_registered_log)
    
    # 3. Clip negatives (Safety)
    pred_casual = np.maximum(pred_casual, 0)
    pred_registered = np.maximum(pred_registered, 0)
    
    # 4. Combine
    pred_total_count = pred_casual + pred_registered
    
    # 5. Get Actual Count
    # We assume 'count' column exists in train_data. 
    # If not, reconstruct it from casual + registered
    if 'count' in train_data.columns:
        actual_count = train_data['count']
    else:
        actual_count = train_data['casual'] + train_data['registered']
    
    # 6. Calculate RMSLE
    # RMSLE = sqrt(mean((log(p+1) - log(a+1))^2))
    # Sklearn's mean_squared_log_error does the log part for us
    rmsle = np.sqrt(mean_squared_log_error(actual_count, pred_total_count))
    
    print(f"---------------------------------------------------")
    print(f"Combined Train RMSLE: {rmsle:.5f}")
    print(f"---------------------------------------------------")
    
    return rmsle, pred_total_count

# --- Usage ---
# Make sure your predictors and train_data are loaded
score, preds = evaluate_on_train(casual_predictor, registered_predictor, train_data)

Generating predictions on Training Set...
---------------------------------------------------
Combined Train RMSLE: 0.16686
---------------------------------------------------
